In [1]:
import pyspark
import os
from pyspark.sql.types import *
from pyspark.sql import Row
import itertools

ModuleNotFoundError: No module named 'pyspark'

## Please uncomment below line if creation of spark context is required 

In [2]:
##sc = pyspark.SparkContext()

## Please change the below input directory paths and the lemma file path as required

In [3]:
project_directory = "/home/rv/Documents/Spring_Projects/DIC/lab5"
data_directory = project_directory + "/data"
lemmaFile = sc.textFile("file://"+project_directory+"/new_lemmatizer.csv")

fileList = os.listdir(data_directory)
lis = [None]*len(fileList)

# Concatenating files to a single RDD
for i,file in enumerate(fileList):
    temp = sc.textFile("file://"+data_directory+"/"+file)
    if i==0:
        textFile = temp 
    else:
        textFile = textFile.union(temp)


# Loading lemma File


## Creating a Row structure for a RDD containing lineInfo - Text 

In [4]:

Line = Row('lineInfo', 'text')
res = textFile.map(lambda line: Line(*line.split(">")))

## Normalizing the word spelling 

In [5]:


test = res.map(lambda line:(line[0],line[1].replace("j","i").replace("v","u")) )



In [6]:
dic = lemmaFile.map(lambda line:line.strip(',').split(','))
r = dic.collect()

## Creating a dictionary from the lemma file
lemma_map = dict()

for val in r:
    lemma_map[val[0]] = val[1:]
    
## combinations is a mapper function used to convert the data to a key - pair/triplet format 
## ngram variable can be used to change the co-occurrence to n=3 or 3 words co-occurring. 
ngram = 2

def combinations(line):
    k = line[0]
    v = line[1].strip().split()
    temp = v[:]
    include = list()
  
    for val in v:
        if val in lemma_map:
            temp.remove(val)
            include+= lemma_map[val]
  
    l = temp+include
  
    return [(k, v) for v in itertools.combinations(l, ngram)]
  
ch = test.map(combinations).flatMap(lambda x:x)

### The below line is the reducer code used to convert the input key-pairs to the required key-list of co-occurences 

In [7]:

final = ch.map(lambda x: (x[1], [x[0]])).reduceByKey(lambda p,q: p+q)

### Please change the output file directory as required

In [8]:
outputFileDir = "output"
final.saveAsTextFile(outputFileDir)